In [1]:
import sys
import os

os.environ["APPLIED_AI_CONF"] = "../../conf/app.toml"

if os.path.abspath("../..") not in sys.path:
    sys.path.insert(0, os.path.abspath("../.."))

from google.cloud.ml.applied.attributes import attributes
from google.cloud.ml.applied.categories import category
from google.cloud.ml.applied.model import domain_model as m
from google.cloud.ml.applied.images import image_to_text
from google.cloud.ml.applied.marketing import marketing

## Generate Product Categories

Based on any product description and (optional) image, we perform the following:

* Create multimodal embeddings (text and image) from the input description and image
* Perform nearest neighbor search on existing product catalog embeddings in Vertex Vector Search to fetch x nearest product ids
* Lookup existing product categories from the retrieved ids.
* Use an LLM to rank returned categories

In [2]:
category.retrieve_and_rank(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)"
)

CategoryList(values=[['Watches', 'Wrist Watches', 'Timewel Wrist Watches']])

In [3]:
category.retrieve_and_rank(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    image = "gs://genai-product-catalog/flipkart_20k_oct26/dbdac18a8ee5a8a48238b9685c96e90a_0.jpg"
)

CategoryList(values=[['Watches', 'Wrist Watches', 'Timewel Wrist Watches'], ['Watches', 'Wrist Watches', 'Times Wrist Watches']])

## Generate Product Attributes

We utilize an LLM to generate product attributes to further enhance a product listing making it more effective for search and recommendations usecases.

Using the same mechanism above, we can use a product description and an image to generate these attributes

In [4]:
attributes.retrieve_and_generate_attributes(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
)

ProductAttributes(product_attributes=[AttributeValue(attribute_name='Brand', attribute_value='Timewel'), AttributeValue(attribute_name='Model', attribute_value='1100-N1949_S'), AttributeValue(attribute_name='Gender', attribute_value='Women'), AttributeValue(attribute_name='Price', attribute_value='Rs.855'), AttributeValue(attribute_name='Discount', attribute_value='Great'), AttributeValue(attribute_name='Product Type', attribute_value='Analog Watch'), AttributeValue(attribute_name='Shipping', attribute_value='Free'), AttributeValue(attribute_name='COD', attribute_value='Yes'), AttributeValue(attribute_name='Guarantee', attribute_value='30 Day Replacement')])

In [5]:
product_attributes = attributes.retrieve_and_generate_attributes(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    image = "gs://genai-product-catalog/flipkart_20k_oct26/dbdac18a8ee5a8a48238b9685c96e90a_0.jpg"
)
product_attributes

ProductAttributes(product_attributes=[AttributeValue(attribute_name='Brand', attribute_value='Timewel'), AttributeValue(attribute_name='Model', attribute_value='1100-N1949_S'), AttributeValue(attribute_name='Gender', attribute_value='Women'), AttributeValue(attribute_name='Price', attribute_value='Rs.855'), AttributeValue(attribute_name='Discount', attribute_value='Great'), AttributeValue(attribute_name='Product Type', attribute_value='Analog Watch'), AttributeValue(attribute_name='Shipping', attribute_value='Free'), AttributeValue(attribute_name='Cash on Delivery', attribute_value='Yes'), AttributeValue(attribute_name='Replacement Guarantee', attribute_value='30 Days')])

[AttributeValue(attribute_name='Brand', attribute_value='Timewel'),
 AttributeValue(attribute_name='Model', attribute_value='1100-N1949_S'),
 AttributeValue(attribute_name='Gender', attribute_value='Women'),
 AttributeValue(attribute_name='Price', attribute_value='Rs.855'),
 AttributeValue(attribute_name='Discount', attribute_value='Great'),
 AttributeValue(attribute_name='Product Type', attribute_value='Analog Watch'),
 AttributeValue(attribute_name='Shipping', attribute_value='Free'),
 AttributeValue(attribute_name='Cash on Delivery', attribute_value='Yes'),
 AttributeValue(attribute_name='Replacement Guarantee', attribute_value='30 Days')]

## Generate Product Image Attributes
We can also use an LVM to analyze a product image an extract product attributes

In [11]:
test_image = m.ImageRequest(image="gs://genai-product-catalog/flipkart_20k_oct26/dbdac18a8ee5a8a48238b9685c96e90a_0.jpg")
image_to_text.image_to_attributes(test_image)

ProductAttributes(product_attributes=[AttributeValue(attribute_name='Brand', attribute_value='Timewel'), AttributeValue(attribute_name='Model', attribute_value='TW1234'), AttributeValue(attribute_name='Gender', attribute_value='Men'), AttributeValue(attribute_name='Movement', attribute_value='Quartz'), AttributeValue(attribute_name='Case Shape', attribute_value='Round'), AttributeValue(attribute_name='Case Material', attribute_value='Alloy'), AttributeValue(attribute_name='Case Diameter', attribute_value='40mm'), AttributeValue(attribute_name='Dial Color', attribute_value='Silver'), AttributeValue(attribute_name='Band Material', attribute_value='Leather'), AttributeValue(attribute_name='Band Color', attribute_value='Black'), AttributeValue(attribute_name='Band Width', attribute_value='20mm'), AttributeValue(attribute_name='Clasp Type', attribute_value='Buckle'), AttributeValue(attribute_name='Water Resistance', attribute_value='30m'), AttributeValue(attribute_name='Functions', attribut

## Generate Marketing Description
We use the product description as well as the above generated product attributes to generate a product marketing description

In [10]:
marketing_request = m.MarketingRequest(
    description="Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    attributes=product_attributes.product_attributes
)

marketing_copy = marketing.generate_marketing_copy(marketing_request)
print(marketing_copy.text)

 **Timewel 1100-N1949_S Analog Watch: A Stylish Timepiece for Women**

Crafted with precision and designed for modern women, the Timewel 1100-N1949_S Analog Watch is a must-have accessory for your collection. This elegant timepiece combines classic design elements with contemporary style, making it a versatile companion for any occasion.

**Key Features:**

- **Analog Display:** Experience the timeless charm of a traditional analog display, featuring easy-to-read hour and minute hands.

- **Durable Construction:** Crafted from high-quality materials, this watch ensures long-lasting durability and resilience against everyday wear and tear.

- **Comfortable Fit:** Designed with comfort in mind, the watch features a comfortable strap that perfectly contours to your wrist.

- **Versatile Design:** With its sleek and sophisticated design, this watch effortlessly complements both casual and formal attire, making it suitable for a wide range of occasions.

- **Affordable Luxury:** Priced at a